In [3]:
import os
import torch
import numpy as np
from glob import glob

def extract_feature_tensor(sample):
    """Extrahiert das Feature-Matrix-Tensor aus einem Sample (tuple, dict oder direkt)."""
    if isinstance(sample, tuple):
        x = sample[0]
    elif isinstance(sample, dict):
        x = sample.get("features", None)
        if x is None:
            raise ValueError("Kein 'features'-Key im Dictionary-Sample.")
    else:
        x = sample

    if isinstance(x, np.ndarray):
        x = torch.tensor(x, dtype=torch.float32)
    elif not torch.is_tensor(x):
        raise TypeError(f"Unbekannter Datentyp: {type(x)}")
    return x

# === Verzeichnisse ===
input_root = "data_new_window/"
output_root_spectral = "data_new_window/spectral_features/"
output_root_temporal = "data_new_window/temporal_features/"

os.makedirs(output_root_spectral, exist_ok=True)
os.makedirs(output_root_temporal, exist_ok=True)

configs = [f for f in os.listdir(input_root) if os.path.isdir(os.path.join(input_root, f))]
print(len(configs))
for config in configs:
    input_dir = os.path.join(input_root, config)
    out_spec = os.path.join(output_root_spectral, config)
    out_temp = os.path.join(output_root_temporal, config)
    os.makedirs(out_spec, exist_ok=True)
    os.makedirs(out_temp, exist_ok=True)

    pt_files = sorted(glob(os.path.join(input_dir, "*.pt")))

    for file_path in pt_files:
        try:
            samples = torch.load(file_path)
            # Falls es sich nicht um eine Liste handelt, mache eine Liste daraus
            if not isinstance(samples, list):
                samples = [samples]

            spectral_samples = []
            temporal_samples = []

            for sample in samples:
                # Entpacke Sample
                if isinstance(sample, tuple):
                    feature_matrix, label, eeg_id, *rest = sample
                elif isinstance(sample, dict):
                    feature_matrix = sample.get("features")
                    label = sample.get("label")
                    eeg_id = sample.get("eeg_id")
                    rest = []
                else:
                    raise ValueError("Unbekanntes Sample-Format")

                # In Tensor umwandeln (falls nötig)
                if isinstance(feature_matrix, np.ndarray):
                    feature_matrix = torch.tensor(feature_matrix, dtype=torch.float32)

                # Split mit alter Feature-Reihenfolge
                spectral = torch.cat([feature_matrix[:, :10], feature_matrix[:, 13:14]], dim=1)
                temporal = torch.cat([feature_matrix[:, 10:13], feature_matrix[:, 14:15]], dim=1)

                # Originalstruktur beibehalten
                spectral_samples.append((spectral, label, eeg_id, *rest))
                temporal_samples.append((temporal, label, eeg_id, *rest))

            # Speichern
            base_name = os.path.basename(file_path)
            torch.save(spectral_samples, os.path.join(out_spec, base_name))
            torch.save(temporal_samples, os.path.join(out_temp, base_name))

            #print(f"{base_name} in {config} erfolgreich aufgeteilt.")

        except Exception as e:
            print(f"Fehler bei {file_path}: {e}")

4


In [ ]:
import torch
import os

path = "montage_datasets/spectral_only/win4_step1/"  # or temporal_only/...

# Load one sample
file = os.listdir(path)[0]
data = torch.load(os.path.join(path, file))

print(f"Loaded {file}, contains {len(data)} samples")

# Inspect the first sample
sample = data[0]
features = sample[0]
label = sample[1]

# Show types and shapes
print("Feature shape:", features.shape)
print("Label:", label)
print("Feature type:", type(features))
print("Label type:", type(label))

In [ ]:
data_folder = "montage_datasets/win4_step1/"
file_paths = sorted(glob(os.path.join(data_folder, "*.pt")))
print(len(file_paths))

In [ ]:
import os
import torch
import numpy as np
from glob import glob

def extract_feature_tensor(sample):
    """Extrahiert das Feature-Matrix-Tensor aus einem Sample (tuple, dict oder direkt)."""
    if isinstance(sample, tuple):
        x = sample[0]
    elif isinstance(sample, dict):
        x = sample.get("features", None)
        if x is None:
            raise ValueError("Kein 'features'-Key im Dictionary-Sample.")
    else:
        x = sample

    if isinstance(x, np.ndarray):
        x = torch.tensor(x, dtype=torch.float32)
    elif not torch.is_tensor(x):
        raise TypeError(f"Unbekannter Datentyp: {type(x)}")
    return x

# === Verzeichnisse ===
input_root = "montage_datasets/win4_step1"
output_root_spectral = "montage_datasets/spectral_only_small/"
output_root_temporal = "montage_datasets/temporal_only_small/"

os.makedirs(output_root_spectral, exist_ok=True)
os.makedirs(output_root_temporal, exist_ok=True)



pt_files = sorted(glob(os.path.join(input_dir, "*.pt")))

for file_path in pt_files:
try:
    samples = torch.load(file_path)
    spectral_samples = []
    temporal_samples = []

    for sample in samples:
        # Entpacke Sample
        if isinstance(sample, tuple):
            feature_matrix, label, eeg_id, *rest = sample
        elif isinstance(sample, dict):
            feature_matrix = sample.get("features")
            label = sample.get("label")
            eeg_id = sample.get("eeg_id")
            rest = []
        else:
            raise ValueError("Unbekanntes Sample-Format")

        # In Tensor umwandeln (falls nötig)
        if isinstance(feature_matrix, np.ndarray):
            feature_matrix = torch.tensor(feature_matrix, dtype=torch.float32)

        # Split mit alter Feature-Reihenfolge
        spectral = torch.cat([feature_matrix[:, :10], feature_matrix[:, 13:14]], dim=1)
        temporal = torch.cat([feature_matrix[:, 10:13], feature_matrix[:, 14:15]], dim=1)

        # Originalstruktur beibehalten
        spectral_samples.append((spectral, label, eeg_id, *rest))
        temporal_samples.append((temporal, label, eeg_id, *rest))

    # Speichern
    base_name = os.path.basename(file_path)
    torch.save(spectral_samples, os.path.join(out_spec, base_name))
    torch.save(temporal_samples, os.path.join(out_temp, base_name))

    #print(f"{base_name} in {config} erfolgreich aufgeteilt.")

except Exception as e:
    print(f"Fehler bei {file_path}: {e}")

In [ ]:
# Code zum Aufteilen und Gruppieren der Features
import os
import torch
import numpy as np

# Eingabe- und Ausgabeordner
OLD_DIR = "data_new_window/win4_step1"
NEW_DIR = "data_new_window/win4_step1_groups"
os.makedirs(NEW_DIR, exist_ok=True)

def split_features_to_channels(sample):
    # sample: numpy array (6, 15)
    features_t = sample.T  # (15, 6)

    groups = [
        features_t[0:5],    # (5, 6)
        features_t[5:10],   # (5, 6)
        features_t[10:13],  # (3, 6)
        features_t[13:15],  # (2, 6)
    ]

    max_len = max(g.shape[0] for g in groups)  # 5 in deinem Fall
    n_channels = groups[0].shape[1]            # 6

    padded_groups = []
    for g in groups:
        pad_size = max_len - g.shape[0]
        if pad_size > 0:
            # Padding unten mit 0en (pad_dim = (0,0,0,pad_size) für 2D Tensor)
            pad = np.zeros((pad_size, n_channels), dtype=g.dtype)
            g_padded = np.vstack([g, pad])
        else:
            g_padded = g
        padded_groups.append(torch.tensor(g_padded, dtype=torch.float32))

    # Nun stapeln entlang der Gruppen-Dimension (4, max_len=5, 6)
    return torch.stack(padded_groups, dim=0)

def process_and_save_files(old_dir, new_dir):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    for filename in os.listdir(old_dir):
        if not filename.endswith(".pt"):
            continue
        filepath = os.path.join(old_dir, filename)
        data = torch.load(filepath)  # data ist Liste von Tuples

        xs = []
        ys = []
        eeg_ids = []
        timestamps = []

        for sample in data:
            x_np, y, eeg_id, timestamp = sample
            # x_np ist numpy array (6, 15)
            x_grouped = split_features_to_channels(x_np)  # Shape (4, group_features, 6)
            xs.append(x_grouped)
            ys.append(y)
            eeg_ids.append(eeg_id)
            timestamps.append(timestamp)

        x_tensor = torch.stack(xs)  # Shape (N, 4, group_features, 6)
        y_tensor = torch.tensor(ys)
        # eeg_ids und timestamps bleiben Listen

        new_data = {
            "x": x_tensor,
            "y": y_tensor,
            "eeg_id": eeg_ids,
            "timestamp": timestamps,
        }

        save_path = os.path.join(new_dir, filename)
        torch.save(new_data, save_path)
        print(f"Gespeichert: {save_path}")


process_and_save_files(OLD_DIR, NEW_DIR)
